# Facebbok Data Miner

This notebook is about showcasing the features of this python application. 
It has a secondary mission, that is by exploring possibilities in this notebook I get insights for designing the interface of the CLI of this app, since it is not totally clear yet, how data relates, and exactly which information can be accessed where.

## Introduction

One day I downloaded my facebook data, and it turned out it's 8.2GBs. That's a lot. Like how is that even possible. I started looking into directories, and found same interesting media files, chat messages and posts. I wanted to dive deeper, but I knew I can't scale it after some point.

Then I googled `facebook data analyzer github`, so that I can just download a package and do the analysis. As you can see in this [list](https://github.com/topics/facebook-data-analyzer) there is not a lot. 

There is a nice library written in Ruby with more than 100 commits and 505 stars, but it's more than 2 years old now. The other thing is that I was looking for a solution with which I can mess around (e.g. maybe use the data getters to do some deep learning on my message history) and I didn't really want to get into Ruby (sorry Ruby users).

There are also some python packages, but they offer little-to-no data interfaces. Most of them focus on visualizing the results with plots, and the data getters were desgined around the plotter functions. Although you may want to check out [fviz](https://github.com/itzmeanjan/fviz) which offers really nice plots. Probably nicer then this library would ever have. 

## Purpose

Then I decided that I would write my own library for this, amongst others for two main reasons:

1. I want the insights from my facebook data, and I want to have really specific insights.
2. I want to improve my Python and get expertise in Python's data science libraries. Gaining some more git and GitHub, software design and project planning skills would also not hurt.


In [ ]:
## Let's dive into